In [1]:
from selenium import webdriver
import time
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as _BeautifulSoup

In [2]:
import csv

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
def write_data(bs4_page, state):
     with open(state + ".csv", "a", newline="", encoding="utf-8") as csv_file:
        fieldnames = ['Artisan Name', 'Craft', 'Village', 'District', 'State']
        writer = csv.DictWriter(csv_file, fieldnames = fieldnames)
        writer.writeheader()
        dict = {}
        for row in bs4_page.find_all('table')[1].find_all('tr'):
            if row == bs4_page.find_all('table')[1].find_all('tr')[-1] or bs4_page.find_all('table')[1].find_all('tr').index(row) == 0:
                continue
            else:
                details = row.find_all('td')

                try:
                    dict['Artisan Name'] = details[1].text
                    dict['Craft'] = details[5].text
                    dict['Village'] = details[7].text
                    dict['District'] = details[10].text
                    dict['State'] = details[11].text
                except:
                    continue  #last line is added because there is a tr inside tr
                writer.writerow(dict)

In [6]:
def scrape(state):
    url = 'http://handicrafts.nic.in/ArtisanData.aspx?MID=SZmOd%2fCrxTo9CHD2XKF+pA%3d%3d'
    browser = webdriver.Chrome(r'C:/Users/DELL/Downloads/chromedriver.exe')
    browser.get(url)


    select = Select(browser.find_element_by_id('ContentPlaceHolder1_ddlSelectionType'))
    select.select_by_value('State Name')

    select = Select(browser.find_element_by_id('ContentPlaceHolder1_DDL_State_new'))
    select.select_by_value(state)
    doc = browser.find_elements_by_xpath('//*[@id="ContentPlaceHolder1_statesubmit_new"]')[0]
    doc.click()

    page_source = browser.page_source

    bs4_page = _BeautifulSoup(page_source, features="html.parser")
    write_data(bs4_page, state)
    number_of_pages = len(bs4_page.find_all('table')[1].find_all('tr')[-1].find_all('td'))
    if number_of_pages > 10:
        for i in range(2, number_of_pages-1):
            doc = browser.find_elements_by_xpath('/html/body/form/div[8]/div/div[1]/div/div[2]/div/table[2]/tbody/tr[202]/td/table/tbody/tr/td[{}]/a'.format(i))[0]
            doc.click()
            page_source = browser.page_source
            bs4_page = _BeautifulSoup(page_source, features="html.parser")
            write_data(bs4_page, state)
    else: 
        for i in range(2, number_of_pages+1):
            doc = browser.find_elements_by_xpath('/html/body/form/div[8]/div/div[1]/div/div[2]/div/table[2]/tbody/tr[202]/td/table/tbody/tr/td[{}]/a'.format(i))[0]
            doc.click()
            page_source = browser.page_source
            bs4_page = _BeautifulSoup(page_source, features="html.parser")
            write_data(bs4_page, state)

    time.sleep(5)
    browser.quit()

In [7]:
states = ['Andaman and Nicobar', 'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Puducherry', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal']
for state in states:
    scrape(state)


### To find the list of states

In [11]:
# states = []
# for state in bs4_page.find_all('select')[1].find_all('option'):
#     states.append(state.text)
# states.pop(0)